In [6]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import sys
sys.path.append('../.')
import datasets

In [14]:
test_size = 0.5
RAND_ST = None


X, y = datasets.load_CIFAR10()

# scaler = StandardScaler()
# scaler.fit(X)
# X = scaler.transform(X)

model_range = 100

print(X.shape)
print(y.shape)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

ValueError: cannot reshape array of size 184320000 into shape (60000,1024)

In [8]:
class nn_dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).to(torch.float32).to(device)
        self.y = torch.from_numpy(y).to(torch.float32).to(device)
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]
        
        return x, y

class neural_network(nn.Module):
    def __init__(self, neuron):
        super().__init__()
        
        self.network = nn.Sequential(
            nn.Linear(784, neuron),
            nn.Linear(neuron, 1),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.network(x)

In [9]:
lr = 0.001
max_epoch = 100

def train_model(neuron, train_loader, test_loader):
    # loss function type
    loss_fn = nn.MSELoss()
    # model
    lr_model = neural_network(neuron).to(device)
    # optimizer type
    optimizer = torch.optim.SGD(lr_model.parameters(), lr=lr)
    
    epochs = max_epoch
    for epoch in range(epochs):
        for x, y in train_loader:
            optimizer.zero_grad()
            loss_fn(lr_model(x), y).backward()
            optimizer.step()
    
    total_train_loss = 0
    total_test_loss = 0
    with torch.no_grad():
        lr_model.eval()
        
        for x, y in train_loader:
            total_test_loss += loss_fn(lr_model(x), y)
        
        for x, y in test_loader:
            total_test_loss += loss_fn(lr_model(x), y)
    
    train_loss = total_train_loss / len(train_loader)
    test_loss = total_test_loss / len(test_loader)

    return train_loss, test_loss

In [10]:
train_losses = []
test_losses = []

neurons = [neuron for neuron in range(10, model_range, 2)]

for neuron in neurons:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=RAND_ST, shuffle=True)
    
    train_dataset = nn_dataset(X_train[:4000], y_train[:4000])
    test_dataset = nn_dataset(X_test[:4000], y_test[:4000])
    train_loader = DataLoader(train_dataset, batch_size=1)
    test_loader = DataLoader(test_dataset, batch_size=1)
    
    train_loss, test_loss = train_model(neuron, train_loader, test_loader)
    train_losses.append(float(train_loss))
    test_losses.append(float(test_loss))
    
    # print(float(train_loss))
    # print(float(test_loss))
    print(neuron)
    # print()

plt.xlabel("neuron")
plt.ylabel("loss (MSELoss)")
plt.plot(neurons, train_losses, label = "train loss")
plt.plot(neurons, test_losses, label = "test loss")
plt.legend()
plt.show()

c:\Users\Huiji\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


10
12
14


KeyboardInterrupt: 